## GeoProcessing with GeoDatabases
This chapter is about working with geodatabase to answer some questions and create maps. 
We will build a crime database

## A crime dashboard

To build a crime dashboard, we will need to collect data to build a database

## Creating the tables
We will create 3 tables to hold the crime data: 
- Area Commands
- Beats
- Incidents

In [10]:
import psycopg2
import requests
from shapely.geometry import Point, Polygon, MultiPolygon, mapping
import datetime


In [11]:
connection = psycopg2.connect(database="pythonspatial", user="postgres")
cursor = connection.cursor()

In [3]:
connection.rollback()
cursor.execute("CREATE TABLE IF NOT EXISTS areacommand(id SERIAL PRIMARY KEY, name VARCHAR(20), geom GEOMETRY)")
cursor.execute("CREATE TABLE IF NOT EXISTS beats (id SERIAL PRIMARY KEY, beat VARCHAR(6), agency VARCHAR(3), areacomm VARCHAR(15),geom GEOMETRY)")
cursor.execute("CREATE TABLE IF NOT EXISTS incidents (id SERIAL PRIMARY KEY, address VARCHAR(72), crimetype VARCHAR(255),date DATE, geom GEOMETRY)")
connection.commit()

### Populating the data

In [56]:
with open('data/albaquque crime.txt', 'r') as file:
    data = json.load(file)
    for acmd in data["features"]: 
        polys = []
        for ring in acmd["geometry"]["rings"]: 
            polys.append(Polygon(ring))
            p=MultiPolygon(polys)    
            name=acmd['attributes']['Area_Command']
            cursor.execute("INSERT INTO areacommand (name, geom) VALUES ('{}',ST_GeomFromText('{}'))".format(name, p.wkt))  
            
connection.commit()


### Beats Data

In [60]:
with open('data/albaquque_beats.txt', 'r') as file:
    data = json.load(file)
    
    for acmd in data['features']:    
        polys=[]    
        for ring in acmd['geometry']['rings']:        
            polys.append(Polygon(ring))    
            p=MultiPolygon(polys)     
            beat = acmd['attributes']['BEAT']    
            agency = acmd['attributes']['AGENCY']    
            areacomm = acmd['attributes']['AREA_COMMA']    
            cursor.execute("INSERT INTO beats (beat, agency,areacomm,geom) VALUES ('{}','{}','{}',    ST_GeomFromText('{}'))".format(beat,agency,areacomm,p.wkt)) 
        
connection.commit()



### Incident Data

In [117]:
from datetime import datetime

with open('data/albaquque_incidents.txt', 'r') as file:
    data = json.load(file)
    
    for a in data["features"][1:]: 
        address=a["attributes"]["BlockAddress"]    
        crimetype=a["attributes"]["IncidentType"]    
        if a['attributes']['ReportDateTime'] is None:        
            pass    
        else:        
            # date = datetime.datetime.fromtimestamp(a['attributes']['ReportDateTime']).date()
            try:
                date = datetime.strptime(a['attributes']['ReportDateTime'], "%Y-%m-%d %H:%M:%S").date()
            except: 
                continue
            try:        
                p=Point(float(a["geometry"]["x"]),float(a["geometry"]["y"]))        
                cursor.execute("INSERT INTO incidents (address,crimetype,date, geom) VALUES        ('{}','{}','{}', ST_GeomFromText('{}'))".format(address,crimetype,str(date), p.wkt))    
            except KeyError:        
                pass
connection.commit()


### Mapping Queries
While we are able to query our data and return the geometry as a well know text, it is not possible to visualize these points. We need to visualize these points on a map, and to do that we need to use ipyleaflet to map the results of our querie on jupyter notebook. 

In [4]:
# !pip install ipyleaflet

In [12]:
import psycopg2
from shapely.geometry import Point,Polygon,MultiPolygon
from shapely.wkb import loads
from shapely.wkt import dumps, loads
import datetime
import json
from ipyleaflet import (Map, Marker, TileLayer, ImageOverlay, Polyline, Polygon, Rectangle, Circle, CircleMarker,    GeoJSON)

In [13]:
cursor.execute("SELECT name, ST_AsGeoJSON(geom) from areacommand")
c = cursor.fetchall()
c[0]

('FOOTHILLS',
 '{"type":"MultiPolygon","coordinates":[[[[-106.519742763,35.050529224],[-106.519741401,35.050529221],[-106.519739522,35.050529218],[-106.518248464,35.05052621],[-106.518299012,35.051733665],[-106.516932057,35.05373802],[-106.516658932,35.054138492],[-106.516657821,35.054140122],[-106.519699412,35.054165487],[-106.519702694,35.054165514],[-106.519714301,35.052815839],[-106.519725571,35.051505216],[-106.519734092,35.051021357],[-106.519742716,35.050531888],[-106.519742763,35.050529224],[-106.519746025,35.05052925],[-106.521781927,35.050531941],[-106.524079969,35.050535018],[-106.528456115,35.050541132],[-106.528461493,35.050156445],[-106.528506972,35.046798773],[-106.530225709,35.046808125],[-106.530226393,35.046808128],[-106.530771249,35.046811088],[-106.532760113,35.046825536],[-106.532760295,35.046878256],[-106.532756223,35.048201237],[-106.532731801,35.049289763],[-106.532713962,35.049735929],[-106.5327057,35.050150205],[-106.5327015,35.050337428],[-106.532750706,35.05

Now that we have a geojson, we can create a map to display it. 

In [14]:
center = [35.106196,-106.629515]
zoom =  13 
map = Map(center=center, zoom=zoom)
map

Map(center=[35.106196, -106.629515], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

- The GeoJSON of the area commands is stored in the variable c. For every item c[x], the GeoJSON is in position 
c[x][1]. The following code will iterate through c and add the GeoJSON to the map.

In [15]:
for x in c: 
    layer = json.loads(x[1])
    layergeojson = GeoJSON(data=layer)
    map.add_layer(layergeojson)

### Incidents by date 

In [16]:
d = datetime.datetime.strptime('2024106','%Y%m%d').date()
query = "SELECT address,crimetype,date, ST_AsGeoJSON(geom) from public.incidents where date='{}' LIMIT 10".format(str(d))
cursor.execute(query)
incidents = cursor.fetchall()
for x in incidents:
    layer = json.loads(x[3])
    layergeojson = GeoJSON(data=layer)
    map.add_layer(layergeojson)

## Incidents in a polygon
Our crime database has a polygon area - area commands and beats- as well as points. To build a crime dashboard, we want to be able to map incidents within a specific area command or beat. We can do that by using JOIN and ST_Intersects. 

In [17]:
connection.rollback()
query = "SELECT ST_AsGeoJSON(i.geom) from incidents i JOIN areacommand acmd ON ST_Intersects(acmd.geom,i.geom)\
WHERE acmd.name like 'FOOTHILLS' and date>=NOW() - interval '10 day' limit 10; "
cursor.execute(query)
incidents = cursor.fetchall()
for x in incidents: 
    layer = json.loads(x[0])
    layergeojson = GeoJSON(data=layer)
    map.add_layer(layergeojson)

In [18]:
query = """
SELECT ST_AsGeoJSON(i.geom) 
FROM incidents i 
JOIN beats b ON ST_Intersects(b.geom, i.geom) 
WHERE b.beat IN ('336', '523', '117', '226', '638', '636') 
  AND date >= NOW() - interval '10 day';
"""
cursor.execute(query)
crime=cursor.fetchall()
for x in crime:    
    layer=json.loads(x[0])    
    layergeojson=GeoJSON(data=layer)    
    map.add_layer(layergeojson)

## Buffers

In [22]:
center = [35.106196,-106.629515]
zoom =  10
map = Map(center=center, zoom=zoom)
map

Map(center=[35.106196, -106.629515], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

We first create a point

In [23]:
from shapely.geometry import mapping 
p = Point([-106.578677,35.062485])
pgeojson = mapping(p)
player = GeoJSON(data=pgeojson)
map.add_layer(player)

NOTE: PostGIS allows us to send data to the database and get data back, none of which has to be in a table. For example: 

In [29]:
connection.rollback()
cursor.execute("SELECT ST_AsGeoJSON(ST_Buffer(ST_GeomFromText('{}')::geography,1500));".format(p.wkt))
buff = cursor.fetchall()
buffer = json.loads(buff[0][0])
bufferlayer  = GeoJSON(data=buffer)
map.add_layer(bufferlayer)

We now have a polygon that we can use to select incidents from

In [43]:
p2 = Point([-106.646582085,35.09257477])

In [54]:
(buff[0][0]).wkt

AttributeError: 'str' object has no attribute 'wkt'

In [57]:
connection.rollback()
cursor.execute("SELECT ST_AsGeoJSON(incidents.geom) FROM incidents where ST_Intersects(ST_GeomFromText('{}'), incidents.geom);".format(p.wkt))
crime=cursor.fetchall()
print(crime)
# for x in crime:    
#     layer=json.loads(x[0])    
#     layergeojson=GeoJSON(data=layer)    
#     map.add_layer(layergeojson)



[]


## Nearest Neighbor
Using buffer, you can get all the incidents within a specified radius of the point of interest. But what if you only want the 5,10. or 15 closest incidents? To do that you can use the <-> operator or k-nearest neighbor


In [61]:
center = [35.106196,-106.629515]
zoom =  10
map = Map(center=center, zoom=zoom)
map

Map(center=[35.106196, -106.629515], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [62]:
connection.rollback()
p = Point([-106.578677,35.062485])
cursor.execute("SELECT ST_AsGeoJSON(incidents.geom), ST_Distance(incidents.geom::geography,ST_GeometryFromText('{}')::geography) from incidents ORDER BY incidents.geom<->ST_GeometryFromText('{}') LIMIT 15".format(p.wkt,p.wkt))
c=cursor.fetchall()
for x in c:    
    layer=json.loads(x[0])    
    layergeojson=GeoJSON(data=layer)    
    map.add_layer(layergeojson)
